In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats.stats import pearsonr   

In [27]:
def convert_dt(original_dt, dt_format):
    original = original_dt
    date_objects = []

    for d in original:
        dt_obj = datetime.strptime(d, dt_format)
        date_objects.append(dt_obj)
        
    return date_objects

In [28]:
taxi_df = pd.read_csv('Taxi_df.csv')
taxi_df = taxi_df.drop('Unnamed: 0', axis = 1)

pudt = taxi_df['pickup_datetime']
dt_convt = []

for dt in pudt: 
    dt_obj = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
    dt_convt.append(dt_obj)
    
taxi_df['pickup_datetime'] = dt_convt

In [29]:
taxi_df.head()

,hack_license,pickup_datetime,payment_type,total_amount,tip_amount,tip_percentage,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_rate
0,A9AE329EA1138052DAC8FDFD8BA86603,2013-03-08 21:55:27,CSH,11.5,0.0,0.000000,2,631,2.5,-73.959610,40.715942,-73.963417,40.692169,Low
1,A9AE329EA1138052DAC8FDFD8BA86603,2013-03-09 18:51:52,CRD,12.0,2.0,16.666667,2,666,1.8,-73.973373,40.763958,-73.977303,40.782139,Mid
2,A9AE329EA1138052DAC8FDFD8BA86603,2013-03-09 19:04:35,CSH,18.5,0.0,0.000000,2,1401,4.4,-73.977837,40.782818,-74.000893,40.737076,Low
3,A9AE329EA1138052DAC8FDFD8BA86603,2013-03-09 20:33:06,CSH,12.5,0.0,0.000000,1,751,2.8,-73.980331,40.784382,-73.984291,40.748688,Low
4,A9AE329EA1138052DAC8FDFD8BA86603,2013-03-09 23:10:38,CRD,8.4,1.4,16.666667,1,261,1.5,-73.986687,40.749474,-73.990677,40.762524,Mid


In [30]:
weather = pd.read_csv('1345853.csv')

date = weather['DATE']
date_objects = []

for d in date:
    dt_obj = datetime.strptime(d, '%m/%d/%Y')
    date_objects.append(dt_obj)
    
weather['DATE'] = date_objects

In [31]:
weather.head()

,STATION,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,SNOW,TAVG,TSUN
0,US1NJBG0023,41.01910,-74.23340,149.4,2013-01-01,0.0,0.0,NaN,NaN
1,US1NJES0011,40.86480,-74.21570,101.8,2013-01-01,0.0,NaN,NaN,NaN
2,USC00280907,40.89174,-74.39635,85.3,2013-01-01,0.0,0.0,NaN,NaN
3,US1NJES0015,40.85650,-74.20040,113.4,2013-01-01,0.0,0.0,NaN,NaN
4,US1NJMD0043,40.55540,-74.27400,4.0,2013-01-01,0.0,0.0,NaN,NaN


In [32]:
grouped = weather.groupby('DATE')
tavg = grouped['TAVG'].agg(np.mean)

missing_w = pd.read_csv('missing_weather.csv')

date_conv = convert_dt(missing_w['date'], '%m/%d/%Y')
missing_w['date'] = date_conv

missing_avg = missing_w[['date', 'avg_temp']]
missing_avg = missing_avg.set_index('date')
missing_avg.columns = ['TAVG']

tavg = tavg.to_frame()

fin_avg = tavg.dropna()
fin_avg = fin_avg.append(missing_avg)

In [33]:
wavg = grouped[['PRCP','SNOW']].agg(np.mean)


wavg['total_precipitation'] = wavg['PRCP'] + (wavg['SNOW'] * 2)

weather_final = fin_avg.join(wavg)


In [34]:
weather_final.head()

,TAVG,PRCP,SNOW,total_precipitation
2013-04-01,49.666667,0.157213,0.0,0.157213
2013-04-02,39.666667,0.015714,0.0,0.015714
2013-04-03,40.000000,0.000000,0.0,0.000000
2013-04-04,40.333333,0.000000,0.0,0.000000
2013-04-05,48.333333,0.002500,0.0,0.002500


In [35]:
day = []

for d in taxi_df['pickup_datetime']:
    day.append(d.date())

taxi_df['day'] = day
taxi_df = taxi_df.set_index('day')

In [37]:
taxi_df = taxi_df.join(weather_final)

In [38]:
taxi_df.head()

,hack_license,pickup_datetime,payment_type,total_amount,tip_amount,tip_percentage,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_rate,TAVG,PRCP,SNOW,total_precipitation
2013-01-01,1A6CAB3092DCB1821793159CE85E889B,2013-01-01 16:57:00,CSH,21.00,0.00,0.000000,5,960,6.49,-73.862747,40.769035,-73.924576,40.739918,Low,32.0,0.000755,0.0,0.000755
2013-01-01,1A6CAB3092DCB1821793159CE85E889B,2013-01-01 13:53:00,CSH,8.50,0.00,0.000000,5,360,2.09,-73.989174,40.774166,-74.000671,40.750278,Low,32.0,0.000755,0.0,0.000755
2013-01-01,1A6CAB3092DCB1821793159CE85E889B,2013-01-01 15:16:00,CRD,12.38,2.38,19.224556,5,540,2.25,-74.004433,40.722157,-73.991508,40.748821,Mid,32.0,0.000755,0.0,0.000755
2013-01-01,1A6CAB3092DCB1821793159CE85E889B,2013-01-01 17:53:00,CSH,4.00,0.00,0.000000,5,60,0.53,-73.984528,40.728703,-73.987152,40.732422,Low,32.0,0.000755,0.0,0.000755
2013-01-01,1A6CAB3092DCB1821793159CE85E889B,2013-01-01 17:57:00,CRD,8.50,1.00,11.764706,5,420,1.39,-73.988556,40.737137,-73.975616,40.750626,Low,32.0,0.000755,0.0,0.000755


In [39]:
taxi_df.to_csv('Taxi_df.csv')